In [ ]:
new['correlation'] = pd.factorize(new['correlation'])[0]

In [1]:
import pandas as pd

In [2]:
testdf=pd.read_csv("C:/Users/salta/Documents/ra/spectrograms/all_test/all_test.csv",dtype=str)
traindf=pd.read_csv("C:/Users/salta/Documents/ra/spectrograms/all_train/all_train.csv",dtype=str)

In [6]:
#testdf (669 rows × 7 columns)
# traindf (3729 rows × 7 columns)
testdf['correlation'] = pd.factorize(testdf['correlation'])[0]
traindf['correlation'] = pd.factorize(traindf['correlation'])[0]

In [9]:
#traindf

In [10]:
import keras
from keras.models import Sequential
#Import from keras_preprocessing not from keras.preprocessing
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import pandas as pd
import numpy as np

In [15]:
datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.25)
train_generator=datagen.flow_from_dataframe(
dataframe=traindf,
directory="C:/Users/salta/Documents/ra/spectrograms/all_train/",
x_col="name",
y_col="correlation",
#subset="training",
batch_size=32,
seed=42,
shuffle=True,
class_mode="raw",
target_size=(32,32))

valid_generator=datagen.flow_from_dataframe(
dataframe=traindf,
directory="C:/Users/salta/Documents/ra/spectrograms/all_train/",
x_col="name",
y_col="correlation",
subset="validation",
batch_size=32,
seed=42,
shuffle=True,
class_mode="raw",
target_size=(32,32))

test_datagen=ImageDataGenerator(rescale=1./255.)
test_generator=test_datagen.flow_from_dataframe(
dataframe=testdf,
directory="C:/Users/salta/Documents/ra/spectrograms/all_test/",
x_col="name",
y_col=None,
batch_size=32,
seed=42,
shuffle=False,
class_mode=None,
target_size=(32,32))

Found 3729 validated image filenames.
Found 932 validated image filenames.
Found 669 validated image filenames.


In [16]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(32,32,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))
model.compile(optimizers.RMSprop(learning_rate=0.0001, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])

In [17]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

In [18]:
model.fit(train_generator,
          steps_per_epoch=STEP_SIZE_TRAIN,
          validation_data=valid_generator,
          validation_steps=STEP_SIZE_VALID,
          epochs=10)

Epoch 1/10


ValueError: in user code:

    C:\Users\salta\anaconda3\lib\site-packages\keras\engine\training.py:830 train_function  *
        return step_function(self, iterator)
    C:\Users\salta\anaconda3\lib\site-packages\keras\engine\training.py:813 run_step  *
        outputs = model.train_step(data)
    C:\Users\salta\anaconda3\lib\site-packages\keras\engine\training.py:771 train_step  *
        loss = self.compiled_loss(
    C:\Users\salta\anaconda3\lib\site-packages\keras\engine\compile_utils.py:201 __call__  *
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    C:\Users\salta\anaconda3\lib\site-packages\keras\losses.py:142 __call__  *
        losses = call_fn(y_true, y_pred)
    C:\Users\salta\anaconda3\lib\site-packages\keras\losses.py:246 call  *
        return ag_fn(y_true, y_pred, **self._fn_kwargs)
    C:\Users\salta\anaconda3\lib\site-packages\tensorflow\python\util\dispatch.py:206 wrapper  **
        return target(*args, **kwargs)
    C:\Users\salta\anaconda3\lib\site-pac

In [ ]:
model.evaluate(valid_generator,
steps=STEP_SIZE_TEST)

In [ ]:
traindf['correlation'].value_counts()

In [ ]:
testdf['correlation'].value_counts()

In [ ]:
test_generator.reset()
pred=model.predict(test_generator,
steps=len(test_generator),
verbose=1)

In [ ]:
predicted_class_indices=np.argmax(pred,axis=1)